In [1]:
%load_ext autoreload
%autoreload 
# from dotenv import load_dotenv
import os
import sys

sys.path.append("../")
# load_dotenv()

from utils.notion_connector import NotionConnector



# Constants: set your Notion API details here
api_key = os.environ.get("NOTION_CONNECTION_SECRET")
database_id = os.environ.get("NOTION_INBOX_DB")  # or page/item ID as needed
books_db = os.environ.get("NOTION_BOOKS_DB")  # or page/item ID as needed

api_key="secret_eR4tFTbg7JicddiQykrLsuQsGhTJpwPkdpl90jlrfhD"
database_id="1f49a9ccad3e459bba2243b6d848657e"
books_db="9bffcff0f8574fd0ae3f08e6215720c3"
series_db=""
NOTION_SERIES_DB="72456e9a10ba48babebb7528effab66e"


# define request to Notion API
NotionInboxConnector = NotionConnector(api_key, database_id, "inbox")
NotionBooksConnector = NotionConnector(api_key, books_db, "books")
results = NotionInboxConnector.get_database()
results

KeyboardInterrupt: 